In [3]:
!pip3 install tensorflow==1.15.0

     |████████████████████████████████| 412.3 MB 11 kB/s              
     |████████████████████████████████| 3.8 MB 70.1 MB/s            
     |████████████████████████████████| 126 kB 74.1 MB/s            
     |████████████████████████████████| 503 kB 53.6 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 4.0 MB 50.0 MB/s            
     |████████████████████████████████| 65 kB 5.4 MB/s             
     |████████████████████████████████| 50 kB 1.3 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 42 kB 211 kB/s             
     |████████████████████████████████| 97 kB 1.5 MB/s             
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7538 sha256=9dc2c255f5818ed170f1e9dd069225dbb6cd34691459718b336363362da6d61b
  Stored in directory: /home/ec2-user/.cache/pip/wheels/19/a7/b9/0740c7a3a7d1d348f04823339274b90de25fbcd217b2ee1fbe
  Created wheel for termcolor: filenam

In [4]:
!pip install 'h5py==2.10.0' --force-reinstall

     |████████████████████████████████| 2.9 MB 32.4 MB/s            
     |████████████████████████████████| 14.8 MB 70.4 MB/s            
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 1.3.0 requires botocore<1.20.50,>=1.20.49, but you have botocore 1.23.23 which is incompatible.


In [5]:
import tensorflow as tf
print(tf.__version__)  # This notebook runs on TensorFlow 1.15.x or earlier
tf_framework_version = tf.__version__

1.15.0


In [6]:
from sagemaker import get_execution_role
from sagemaker import Session
role = get_execution_role()
sess = Session()
bucket = sess.default_bucket()

In [7]:
import os
#import keras
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import model_from_json
import h5py

json_file = open('model_final.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = tf.keras.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_final.h5")
print("Loaded model from disk")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Loaded model from disk


In [4]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

# Note: This directory structure will need to be followed 
model_version = '1'
export_dir = 'export/Servo/' + model_version
# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)

# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

from tensorflow.keras import backend as K
with K.get_session() as sess:
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


In [8]:
import tarfile
import sagemaker
model_archive = 'bp-ppg-model.tar.gz'
with tarfile.open(model_archive, mode='w:gz') as archive:
    archive.add('export', recursive=True) 

# upload model artifacts to S3
model_data = sagemaker.Session().upload_data(path=model_archive, key_prefix='BP-PPG-Model')

In [9]:
model_data

's3://sagemaker-us-east-1-522170009381/BP-PPG-Model/bp-ppg-model.tar.gz'

In [13]:
!touch train.py
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = model_data,
                                  role = role,
                                  framework_version = tf_framework_version,
                                  entry_point = 'train.py')
%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.t2.medium' )

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.39 µs
--------!